## Setup VM for use with fast.ai

###Check VM Status

Check RAM allocation

In [4]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 11.9 GB  | Proc size: 140.6 MB
GPU RAM Free: 8983MB | Used: 2456MB | Util  21% | Total 11439MB


Uncomment line to restart Jupyter Kernel if necessary

In [0]:
#!pkill -9 -f ipykernel_launcher

Python version

In [6]:
!python -V

Python 3.6.3


## Introduction to our first task: 'Dogs vs Cats'

Download Dataset if using Google Colab

In [13]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [14]:
!wget http://files.fast.ai/data/dogscats.zip && unzip -q dogscats.zip -d data/

--2018-08-27 10:15:03--  http://files.fast.ai/data/dogscats.zip
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857214334 (818M) [application/zip]
Saving to: ‘dogscats.zip’

dogscats.zip        100%[===================>] 817.50M  86.1MB/s    in 10s     

2018-08-27 10:15:13 (80.7 MB/s) - ‘dogscats.zip’ saved [857214334/857214334]



In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [10]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [0]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [15]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [16]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

94658560/94653016 [==============================] - 2s 0us/step


In [0]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 551s 2s/step - loss: 0.2144 - acc: 0.9440 - val_loss: 0.0611 - val_acc: 0.9803
Epoch 2/3
359/359 [==============================] - 529s 1s/step - loss: 0.0831 - acc: 0.9721 - val_loss: 0.1189 - val_acc: 0.9763
Epoch 3/3
359/359 [==============================] - 520s 1s/step - loss: 0.0719 - acc: 0.9749 - val_loss: 0.1173 - val_acc: 0.9753
CPU times: user 33min 8s, sys: 2min 9s, total: 35min 18s
Wall time: 26min 41s


In [0]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [20]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 576s 2s/step - loss: 0.0845 - acc: 0.9746 - val_loss: 8.0104 - val_acc: 0.4975
CPU times: user 11min 25s, sys: 44.3 s, total: 12min 9s
Wall time: 9min 41s
